In [1]:
#Step 1: Scraping Dependencies
from bs4 import BeautifulSoup
from splinter import Browser
from selenium import webdriver
from selenium.webdriver.common.by import By

#Step 2: Database Dependencies
import pymongo

#Python Dependencies
import pandas as pd
import time


# STEP 1: Web Scraping

1. Use Splinter to create a browser object connected to NASA Mars website
2. Use BeautifulSoup to parse HTML and retrieve information

In [2]:
#define path to chrome driver
executable_path = {'executable_path': '/chromedriver'}

#Set default behaviors
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-notifications")

#Create browser object
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
mars_dict = {}

### NASA Mars News

In [90]:
#Connect to NASA Mars webpage
browser.visit("https://mars.nasa.gov/news")

In [91]:
#Create a soup object 
html = browser.html
soup = BeautifulSoup(html, 'lxml')

In [92]:
#Access the titles and article teasers for each news article
articles = soup.findAll("li", class_="slide")
titles = []
teasers = []

for article in articles:
    title = article.find('div', class_="content_title").text
    titles.append(title)
    teaser = article.find('div', class_="article_teaser_body").text
    teasers.append(teaser)

#Append to the Mars Dictionary
mars_dict.update({"titles": titles, "teasers": teasers})

### JPL Mars Space Images

In [93]:
#Connect to JPL NASA web page
browser.visit("https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars")

In [94]:
#Create a soup object 
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [95]:
#Use Splinter to Click on full JPG image for featured image
browser.find_by_id('full_image').first.click()

In [96]:
pic = browser.links.find_by_partial_text("more info")
pic.click()

In [97]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

img_url = soup.select_one("figure.lede a img").get("src")
feat_img_url = "https://www.jpl.nasa.gov" + img_url
print(feat_img_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA19083_hires.jpg


In [98]:
#Update dictionary with featured image url
mars_dict.update({"Featured_Image_URL": feat_img_url})

### Mars Facts
1. Use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
2. Use Pandas to convert the data to a HTML table string.

In [12]:
url = 'https://space-facts.com/mars'

In [13]:
tables = pd.read_html(url)

In [58]:
#Remove header
mars_profile = tables[0]
mars_profile

,Mars Planet Profile,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [47]:
#Convert Mars Profile table to an html table
mars_html_table = mars_profile.to_html()

#Clean up table
mars_html_table.replace('\n', '')

#Save directly to file
mars_profile.to_html('mars_profile.html')

In [55]:
# #Convert Mars Earth table to an html table
# mars_earth_html_table = mars_earth.to_html()

# #Clean up table
# mars_earth_html_table.replace('\n', '')

# #Save directly to file
# mars_profile.to_html('mars_earth.html')

In [56]:
# !open mars_earth.html

In [ ]:
#Convert table to an html table
mars_profile_tab = mars_profile.to_html()

#Clean up table
mars_table.replace('\n', '')

#Save directly to file
df.to_html('table.html')

In [17]:
    mars_dict.update({"Mars_Facts_Table": mars_table})
    time.sleep(5)

In [19]:
print(mars_table)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Mars Planet Profile</th>
      <th>Value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</

### Mars Hemispheres

In [7]:
# Mars hemisphere name and image to be scraped
main_url = 'https://astrogeology.usgs.gov'
hemis_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
time.sleep(2)

In [8]:
#Create Soup Object
browser.visit(hemis_url)

hemis_html = browser.html
hemis_soup = BeautifulSoup(hemis_html, 'lxml')

In [10]:
# Mars Hemispheres Data Page Links
all_hemis = hemis_soup.find('div', class_='collapsible results')
mars_hemis = all_hemis.find_all('div', class_='item')

hemi_img_urls = []

# Loop through the main hemisphere data page
for hemi in mars_hemis:
    # Collect Title
    hemisphere = hemi.find('div', class_="description")
    title = hemisphere.h3.text
    
    # Scrape image links on hemisphere page
    hemi_link = hemi.a["href"]    
    browser.visit(main_url + hemi_link)
    
    image_html = browser.html
    image_soup = BeautifulSoup(image_html, 'html.parser')
    
    image_link = image_soup.find('div', class_='downloads')
    image_url = image_link.find('li').a['href']

    # Create Dictionary to with key: title and value: url info
    image_dict = {}
    image_dict['title'] = title
    image_dict['img_url'] = image_url
    
    hemi_img_urls.append(image_dict)

mars_dict.update({"image_dict": hemi_img_urls})

In [11]:
print(mars_dict)

{'image_dict': [{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]}


# Step2: MongoDB and Flask Application

In [109]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [ ]:
#Create a MongoDB database
db = client.mars_db

#Add collection
data_collection = db["mars"]

#Insert dictionary into mars_db
data_collection.insert_one(mars_dict)